#### Training a new Model.

We have annotated 500 example with 6 labels in our dataset , it is now time to train it and we will use this model to build the script for the training model.

The plan is to train a custom model for NER, then use prodigy for with that custom model and reannodate the data with the model, and keep training the model with the new anodated data from pretraining.

- https://www.freecodecamp.org/news/getting-started-with-ner-models-using-huggingface/
-  And we will be using the Jean-Baptiste/camembert-ner-with-dates model from huggingface.

### Loading the data.

We have annodated data from prodigy , we will be using a script to load the data  before training our model on the data.

In [36]:
from config import PRODIGY_DATABASE_URL

In [37]:
from urllib.parse import urlparse
result = urlparse(PRODIGY_DATABASE_URL)
username = result.username
password = result.password
database = result.path[1:]
hostname = result.hostname
port = result.port
connection_params = {
    "database": database,
    "user": username,
    "password": password,
    "host": hostname,
    "port": port
}

In [39]:
from prodigy.components.db import connect

In [40]:
database_connection = connect(db_id="postgresql", db_settings=connection_params)

In [41]:
database_connection.datasets

['ner_cong_news',
 'ner_congo_news_log_org_pers_reanodate',
 'sample_news_600',
 'sample-news-with-bert']

In [42]:
drc_new_annotation = database_connection.get_dataset("sample_news_600")

In [46]:
import spacy

In [47]:
nlp = spacy.blank("fr")

In [63]:
from spacy.tokens import Doc
from spacy.vocab import Vocab
from spacy.training import biluo_tags_to_offsets, iob_to_biluo


In [ ]:
# this code convert the data from prodigy anndoation to the data we can use to train a transtofrmer model.

In [87]:
dataset = []
examples = ((eg["text"], eg) for eg in drc_new_annotation)
for doc, eg in nlp.pipe(examples, as_tuples=True):
    if eg.get("spans"):
        doc.ents = [doc.char_span(s["start"], s["end"], s["label"]) for s in eg["spans"]]
    else :
        doc.ents = []
    iob_tags = [f"{t.ent_iob_}-{t.ent_type_}" if t.ent_iob_ else "O" for t in doc]
    iob_tags = [t.strip("-") for t in iob_tags]
    tokens = [str(t) for t in doc]
    biluo_tags = iob_to_biluo(iob_tags)
    offsets = biluo_tags_to_offsets(doc, biluo_tags)
    temp_data = {
        "tokens": tokens,
        "tags": iob_tags,
        "offsets": offsets,
        "bilod_tags": biluo_tags,
    }
    dataset.append(temp_data)

In [88]:
dataset[1]

{'tokens': ['Il',
  'l’',
  'a',
  'dit',
  ',',
  'samedi',
  '26',
  'mars',
  ',',
  'apres',
  'son',
  'election',
  ',',
  'en',
  'presence',
  'des',
  'deputes',
  'provinciaux',
  'et',
  'de',
  'nombreuses',
  'autres',
  'personnalites',
  ',',
  'a',
  'Lubumbashi',
  '.',
  '«A',
  'la',
  'naissance',
  ',',
  'personne',
  'n’',
  'est',
  'venu',
  'avec',
  'de',
  'l’',
  'argent',
  '.'],
 'tags': ['O',
  'O',
  'O',
  'O',
  'O',
  'B-TIME',
  'I-TIME',
  'I-TIME',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'offsets': [(12, 26, 'TIME'), (125, 135, 'LOC')],
 'bilod_tags': ['O',
  'O',
  'O',
  'O',
  'O',
  'B-TIME',
  'I-TIME',
  'L-TIME',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'U-LOC',
  'O',
  'O',

In [3]:
text_data = """
Mais, retenez que l’essentiel de mon programme est le travail», a indique Jean-Claude Kazembe.Le gouverneur du Haut-Katanga a egalement promis de de travailler en collaboration avec la population pour developper cette  nouvelle province.Jean-Claude Kazembe Musonda a remercie les deputes provinciaux pour la confiance lui a accordee, mais surtout pour leur respect envers la population et l’autorite morale de la Majorite presidentielle (MP), Joseph Kabila :«Le depute elu par la population respecte le choix de ses electeurs.
"""

In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner-with-dates")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner-with-dates")


##### Process text sample (from wikipedia)




In [3]:
text_data = """
Le Tout Puissant Mazembe a battu le Football club Lubumbashi-Sport par un score de 3 buts a 1 au stade TP Mazembe de la commune de Kamalondo a Lubumbashi dans le Haut-Katanga, ce mercredi 17 fevrier 2021.C'etait en match de la phase retour de la Ligue Nationale de Football, Vodacom Ligue 1.C'est a la 16eme minute de jeu que les kamikazes ont ouvert le score sur un coup-franc tire par le joueur Wandenga de Lubumbashi-Sport.Moins d'un quart d'heure apres ce but de Lubumbashi-Sport, les corbeaux de Mazembe vont obtenir un penalty a la 30eme minute qui sera transforme par Christian Kouame apres une faute de Kamikazes.Les deux equipes se sont separees a la mi-temps par un but partout.De retour des vestiaires, Adam Bossu va alourdir le score pour le TP Mazembe a la 53eme minute.
"""

In [14]:
from transformers import pipeline

nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")
transformed_data = nlp(text_data)

In [15]:
transformed_data.reverse

[{'entity_group': 'ORG',
  'score': 0.90294963,
  'word': 'Tout Puissant Mazembe',
  'start': 3,
  'end': 25},
 {'entity_group': 'ORG',
  'score': 0.9533042,
  'word': 'Football club Lubumbashi-Sport',
  'start': 36,
  'end': 67},
 {'entity_group': 'LOC',
  'score': 0.9589909,
  'word': 'stade TP Mazembe',
  'start': 97,
  'end': 114},
 {'entity_group': 'LOC',
  'score': 0.9939287,
  'word': 'Kamalondo',
  'start': 131,
  'end': 141},
 {'entity_group': 'LOC',
  'score': 0.9953526,
  'word': 'Lubumbashi',
  'start': 143,
  'end': 154},
 {'entity_group': 'LOC',
  'score': 0.99548435,
  'word': 'Haut-Katanga',
  'start': 162,
  'end': 175},
 {'entity_group': 'DATE',
  'score': 0.97593147,
  'word': 'ce mercredi 17 fevrier 2021',
  'start': 176,
  'end': 204},
 {'entity_group': 'ORG',
  'score': 0.6167741,
  'word': 'Ligue Nationale de Football',
  'start': 246,
  'end': 274},
 {'entity_group': 'MISC',
  'score': 0.62800646,
  'word': 'Vodacom Ligue 1.',
  'start': 275,
  'end': 292},
 {'e

In [82]:
output = tokenizer.encode(text_data)

In [34]:
tokenizer.sep_token

'</s>'

In [83]:
print(output)

[5, 54, 543, 4804, 2914, 16996, 6614, 35, 33, 9252, 16, 12079, 1046, 71, 2509, 19557, 10, 1895, 26, 21154, 37, 23, 3856, 8, 135, 5316, 33, 124, 36, 2623, 18510, 16996, 6614, 35, 8, 13, 928, 8, 11487, 341, 88, 1233, 33, 71, 2509, 19557, 10, 1895, 29, 16, 4860, 26, 707, 622, 16310, 7, 44, 1600, 458, 4152, 16441, 325, 2139, 9, 228, 11, 12821, 22, 1136, 8, 13, 2231, 665, 8, 13, 3593, 4395, 8, 12079, 7, 457, 11808, 310, 3593, 2758, 228, 11, 41, 33, 13, 475, 3141, 3417, 8, 357, 27, 19, 1482, 2900, 1469, 138, 80, 96, 1422, 16, 3856, 32, 23, 392, 26, 16488, 4808, 37, 16, 1954, 692, 8504, 21255, 8, 71, 2509, 19557, 10, 1895, 26, 21154, 9, 7040, 1210, 18, 11, 59, 5207, 18, 11, 1130, 7129, 44, 635, 8, 71, 2509, 19557, 10, 1895, 26, 21154, 7, 19, 27431, 290, 8, 16996, 6614, 35, 774, 1207, 23, 27489, 33, 13, 417, 3141, 3417, 31, 210, 7196, 37, 4012, 16693, 3188, 7129, 28, 2871, 8, 11487, 8250, 138, 80, 9, 1607, 116, 20998, 10, 48, 56, 48, 1244, 35, 80, 33, 13, 1212, 26, 4677, 37, 23, 635, 1397, 9, 

In [62]:
import spacy

In [69]:
import spacy
nlp = spacy.blank("fr") 
config = {
    "model": {
        "@architectures": "spacy-transformers.TransformerModel.v3",
        "name": "Jean-Baptiste/camembert-ner-with-dates",
        "tokenizer_config" : {"use_fast": False}
    }
}
nlp.add_pipe("transformer", config=config)
nlp.initialize() # XXX don't forget this step!# all the Transformer output is stored here

Some weights of the model checkpoint at Jean-Baptiste/camembert-ner-with-dates were not used when initializing CamembertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertModel were not initialized from the model checkpoint at Jean-Baptiste/camembert-ner-with-dates and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
from spacy import displacy

With this we have or dataset with tokens and their corresponding tags , now we can think about building a dataset we will be using to train our ner model.

In [71]:
nlp(text_data).t


Le Tout Puissant Mazembe a battu le Football club Lubumbashi-Sport par un score de 3 buts a 1 au stade TP Mazembe de la commune de Kamalondo a Lubumbashi dans le Haut-Katanga, ce mercredi 17 fevrier 2021.C'etait en match de la phase retour de la Ligue Nationale de Football, Vodacom Ligue 1.C'est a la 16eme minute de jeu que les kamikazes ont ouvert le score sur un coup-franc tire par le joueur Wandenga de Lubumbashi-Sport.Moins d'un quart d'heure apres ce but de Lubumbashi-Sport, les corbeaux de Mazembe vont obtenir un penalty a la 30eme minute qui sera transforme par Christian Kouame apres une faute de Kamikazes.Les deux equipes se sont separees a la mi-temps par un but partout.De retour des vestiaires, Adam Bossu va alourdir le score pour le TP Mazembe a la 53eme minute.

In [72]:
displacy.render(nlp(text_data),  style="ent", jupyter=True)

/Users/es.py/Library/Caches/pypoetry/virtualenvs/deep-learning-nlp-stuff/lib/python3.7/site-packages/spacy/displacy/__init__.py:200: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


The below code is an adaptation from the NER dataset from huggingface.

In [1]:
import spacy
import spacy_wrap

nlp = spacy.blank("en")

config = {
    "doc_extension_trf_data": "clf_trf_data",  # document extention for the forward pass
    "doc_extension_prediction": "sentiment",  # document extention for the prediction
    "labels": ["negative", "neutral", "positive"],
    "model": {
        "name": "cardiffnlp/twitter-roberta-base-sentiment",  # the model name or path of huggingface model
    },
}

transformer = nlp.add_pipe("classification_transformer", config=config)

doc = nlp("spaCy is a wonderful tool")

print(doc._.clf_trf_data)
# TransformerData(wordpieces=...
print(doc._.sentiment)
# 'positive'
print(doc._.sentiment_prob)
#{'prob': array([0.004, 0.028, 0.969], dtype=float32), 'labels': ['negative', 'neutral', 'positive']}

Downloading: 100%|██████████| 747/747 [00:00<00:00, 292kB/s]
Downloading: 100%|██████████| 878k/878k [00:00<00:00, 903kB/s] 
Downloading: 100%|██████████| 446k/446k [00:00<00:00, 634kB/s] 
Downloading: 100%|██████████| 150/150 [00:00<00:00, 85.6kB/s]
Downloading: 100%|██████████| 476M/476M [00:59<00:00, 8.44MB/s] 


TransformerData(wordpieces=WordpieceBatch(strings=[['<s>', 'sp', 'a', 'Cy', 'Ġis', 'Ġa', 'Ġwonderful', 'Ġtool', '</s>']], input_ids=array([[    0,  4182,   102, 25826,    16,    10,  4613,  3944,     2]],
      dtype=int32), attention_mask=array([[1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32), lengths=[9], token_type_ids=None), model_output=ModelOutput([('logits', array([[-2.376998 , -0.316459 ,  3.2421818]], dtype=float32))]), align=Ragged(data=array([[1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7]], dtype=int32), lengths=array([3, 1, 1, 1, 1], dtype=int32), data_shape=(-1,), starts_ends=None))
positive
{'prob': array([0.004, 0.028, 0.969], dtype=float32), 'labels': ['negative', 'neutral', 'positive']}


In [2]:
from spacy_transformers.span_getters import get_doc_spans

In [9]:
import spacy
import spacy_wrap

nlp = spacy.blank("fr")

config = {
    "model": {
        "@architectures": "spacy-transformers.TransformerModel.v3",
        "name": "Jean-Baptiste/camembert-ner-with-dates",
        "tokenizer_config" : {"use_fast": False},
        "get_spans":  {"@span_getters": "spacy-transformers.doc_spans.v1"}
    }
}

transformer = nlp.add_pipe("ner", config=config)

KeyError: 'resize_output'

Upon checking I found that it is nearly not possible to use the spacy teacher feature with a tranformer model such as the BERT model we find fined tuned on the Camembert Model. A solution will to use only the tokenization feature offer by the model and annotate a few samples with the pretrain it with those few sample if we are not happy with the results . But as of now , I can see that the model update with the french wikipedia is doing a good job.

In [ ]:
This failled , we will leverage the cambert model for ner to train our dataset.